# Credit Card Fraud Analysis

In [33]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report

warnings.filterwarnings('ignore')


In [34]:
data = pd.read_csv('creditcard.csv')
data.head()

,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,...,V21,V22,V23,V24,V25,V26,V27,V28,Amount,Class
0,0,-1.359807,-0.072781,2.536347,1.378155,-0.338321,0.462388,0.239599,0.098698,0.363787,...,-0.018307,0.277838,-0.110474,0.066928,0.128539,-0.189115,0.133558,-0.021053,149.62,0.0
1,0,1.191857,0.266151,0.166480,0.448154,0.060018,-0.082361,-0.078803,0.085102,-0.255425,...,-0.225775,-0.638672,0.101288,-0.339846,0.167170,0.125895,-0.008983,0.014724,2.69,0.0
2,1,-1.358354,-1.340163,1.773209,0.379780,-0.503198,1.800499,0.791461,0.247676,-1.514654,...,0.247998,0.771679,0.909412,-0.689281,-0.327642,-0.139097,-0.055353,-0.059752,378.66,0.0
3,1,-0.966272,-0.185226,1.792993,-0.863291,-0.010309,1.247203,0.237609,0.377436,-1.387024,...,-0.108300,0.005274,-0.190321,-1.175575,0.647376,-0.221929,0.062723,0.061458,123.50,0.0
4,2,-1.158233,0.877737,1.548718,0.403034,-0.407193,0.095921,0.592941,-0.270533,0.817739,...,-0.009431,0.798278,-0.137458,0.141267,-0.206010,0.502292,0.219422,0.215153,69.99,0.0


In [35]:
data['Class'].value_counts()

Class
0.0    138458
1.0       264
Name: count, dtype: int64

In [36]:
#dataset info
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 138723 entries, 0 to 138722
Data columns (total 31 columns):
 #   Column  Non-Null Count   Dtype  
---  ------  --------------   -----  
 0   Time    138723 non-null  int64  
 1   V1      138723 non-null  float64
 2   V2      138723 non-null  float64
 3   V3      138723 non-null  float64
 4   V4      138723 non-null  float64
 5   V5      138723 non-null  float64
 6   V6      138723 non-null  float64
 7   V7      138723 non-null  float64
 8   V8      138723 non-null  float64
 9   V9      138723 non-null  float64
 10  V10     138723 non-null  float64
 11  V11     138723 non-null  float64
 12  V12     138723 non-null  float64
 13  V13     138723 non-null  float64
 14  V14     138722 non-null  float64
 15  V15     138722 non-null  float64
 16  V16     138722 non-null  float64
 17  V17     138722 non-null  float64
 18  V18     138722 non-null  float64
 19  V19     138722 non-null  float64
 20  V20     138722 non-null  float64
 21  V21     13

In [37]:
data.isnull().sum()

Time      0
V1        0
V2        0
V3        0
V4        0
V5        0
V6        0
V7        0
V8        0
V9        0
V10       0
V11       0
V12       0
V13       0
V14       1
V15       1
V16       1
V17       1
V18       1
V19       1
V20       1
V21       1
V22       1
V23       1
V24       1
V25       1
V26       1
V27       1
V28       1
Amount    1
Class     1
dtype: int64

In [40]:
data.dropna(inplace=True)

In [43]:
data['Class'].value_counts()

Class
0.0    138458
1.0       264
Name: count, dtype: int64

Highly unbalanced data sets

1 - fraud Transaction <br>
0 - Normal Transaction

In [47]:
# seperating the legit and frad Transaction

legit = data[data.Class == 0]
fraud = data[data.Class == 1]

print(legit.shape)
print(fraud.shape)

(138458, 31)
(264, 31)


In [49]:
# Statistical measures of the data
print(legit.Amount.describe(),end = '\n \n')
print(fraud.Amount.describe())

count    138458.000000
mean         91.326355
std         247.818289
min           0.000000
25%           6.040000
50%          23.990000
75%          81.000000
max       19656.530000
Name: Amount, dtype: float64
 
count     264.000000
mean      115.387008
std       245.189867
min         0.000000
25%         1.000000
50%         9.560000
75%        99.990000
max      1809.680000
Name: Amount, dtype: float64


In [60]:
# compare mean values for both transactions
data.groupby('Class').mean()

,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,...,V20,V21,V22,V23,V24,V25,V26,V27,V28,Amount
Class,,,,,,,,,,,,,,,,,,,,,
0.0,51587.410854,-0.237937,0.007939,0.687613,0.131863,-0.276876,0.080863,-0.106094,0.061851,-0.086793,...,0.041947,-0.041688,-0.117943,-0.033357,0.011940,0.130242,0.023468,-0.000135,0.002096,91.326355
1.0,42420.212121,-5.610819,3.946507,-7.143814,4.512206,-3.977333,-1.480669,-5.936405,1.508791,-2.604923,...,0.239899,1.256981,-0.311690,-0.116543,-0.107463,0.197988,0.056528,0.489643,0.081783,115.387008


In [52]:
# The mean for Class 0 is 51587 and for Class 1 is 42420
# Data Sample is Unbalanced very low = 1 in Class
# we need to use Under-Sampling Technique to fit into the model

### Under-Sampling

Build a sample dataset containing similar distribution of normal transactions and Fraudulent Transactions.


Number of Fradulent Transaction -> 264

In [53]:
legit_sample = legit.sample(n=264)

### Concatenating two Dataframes

In [54]:
new_dataset = pd.concat([legit_sample, fraud], axis=0)


In [55]:
new_dataset.head()

,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,...,V21,V22,V23,V24,V25,V26,V27,V28,Amount,Class
90458,63029,1.177607,0.021177,1.152018,1.250339,-0.728547,0.026787,-0.507635,0.064288,0.783658,...,-0.075608,0.031808,-0.059677,0.059705,0.492316,-0.389955,0.077604,0.036683,9.99,0.0
124126,77185,-0.247299,0.938542,1.042906,0.390816,1.009343,0.252454,1.082187,-0.383255,-0.355176,...,-0.002226,0.437717,-0.386930,-0.835153,-0.131369,-0.331504,-0.049683,-0.235400,16.91,0.0
29363,35440,-0.676848,0.778389,1.315636,0.710040,0.431858,-0.175539,0.965096,-0.066820,-0.737364,...,0.080523,0.054971,-0.280535,-0.459201,0.644853,-0.143911,0.036878,0.073466,78.72,0.0
37567,39014,1.219594,0.179856,-0.452172,0.107321,0.676375,0.234187,0.324035,-0.026272,-0.518040,...,-0.351315,-1.067555,-0.019415,-1.133798,0.359151,0.216701,-0.052464,-0.008495,37.90,0.0
34541,37717,-0.602465,1.298622,1.231384,1.016285,-0.399336,-0.875420,0.409570,0.226321,-0.875351,...,0.328815,0.908882,-0.096103,0.764883,-0.227829,-0.280735,0.061035,0.105577,18.40,0.0


In [56]:
new_dataset.tail()

,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,...,V21,V22,V23,V24,V25,V26,V27,V28,Amount,Class
125342,77627,-7.139060,2.773082,-6.757845,4.446456,-5.464428,-1.713401,-6.485365,3.409395,-3.053493,...,1.303250,-0.016118,-0.876670,0.382230,-1.054624,-0.614606,-0.766848,0.409424,106.90,1.0
128479,78725,-4.312479,1.886476,-2.338634,-0.475243,-1.185444,-2.112079,-2.122793,0.272565,0.290273,...,0.550541,-0.067870,-1.114692,0.269069,-0.020572,-0.963489,-0.918888,0.001454,60.00,1.0
131272,79540,-0.114361,1.036129,1.984405,3.128243,-0.740344,1.548619,-1.701284,-2.203842,-1.242265,...,-1.032935,1.196428,-0.112857,0.254719,0.696668,0.482370,0.129969,0.223924,0.20,1.0
135718,81372,-0.885254,1.790649,-0.945149,3.853433,-1.543510,0.188582,-2.988383,1.344059,-2.294535,...,0.746160,0.550802,-0.034882,-0.567608,-0.528318,0.258782,0.506893,0.176736,7.57,1.0
137705,82289,-1.464897,1.975528,-1.077145,2.819191,0.069850,-0.789044,-1.196101,0.673654,-1.363724,...,0.174099,-0.272505,-0.031549,-0.406166,0.157769,-0.104393,0.073796,-0.041570,1.00,1.0


In [57]:
new_dataset['Class'].value_counts()

Class
0.0    264
1.0    264
Name: count, dtype: int64

In [58]:
new_dataset.groupby('Class').mean()

,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,...,V20,V21,V22,V23,V24,V25,V26,V27,V28,Amount
Class,,,,,,,,,,,,,,,,,,,,,
0.0,52134.375000,-0.084563,-0.027100,0.705802,0.180025,-0.339172,0.080996,-0.160628,0.023719,-0.034298,...,0.044747,-0.086640,-0.118894,-0.016383,0.060586,0.146035,-0.009589,0.025323,0.014893,92.490189
1.0,42420.212121,-5.610819,3.946507,-7.143814,4.512206,-3.977333,-1.480669,-5.936405,1.508791,-2.604923,...,0.239899,1.256981,-0.311690,-0.116543,-0.107463,0.197988,0.056528,0.489643,0.081783,115.387008


In [59]:
# The mean for Class 0 is '52134' close to previous mean 51587 and for Class 1 is 42420 same as prev (same data)

#### Splitting the data into Features & Targets

In [61]:
X = new_dataset.drop(columns='Class', axis=1)
Y = new_dataset['Class']


In [62]:
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.2, stratify=Y, random_state=2)

In [63]:
print(X_train.shape, X_test.shape, X.shape)

(422, 30) (106, 30) (528, 30)


### Model Training
<br>
 Logistic Regression

In [64]:
lr_model = LogisticRegression()

In [65]:
lr_model.fit(X_train, y_train)

LogisticRegression()

#### Model Evaluation

Accuracy Score

In [66]:
# accuracy on training data
X_train_pred = lr_model.predict(X_train)
training_data_accuracy = accuracy_score(X_train_pred, y_train)
print('Accuracy on Training data : ', training_data_accuracy)

Accuracy on Training data :  0.9549763033175356


In [68]:
# accuracy on Test data
X_test_pred = lr_model.predict(X_test)
test_data_accuracy = accuracy_score(X_test_pred, y_test)
print('Accuracy on Test data :', test_data_accuracy)

Accuracy on Test data : 0.9433962264150944
